In [ ]:
import requests
import psycopg2
import json

# Make a GET request to the URL and parse the response as JSON
url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Planning_Primary_Zoning/FeatureServer/0/query?outFields=ZONE_CODE&where=1%3D1&f=geojson'
response = requests.get(url)
data = json.loads(response.content)

features = data['features']
    
conn = psycopg2.connect(database="lab0", user="postgres", password="Homework13", host="34.27.240.213")
cur = conn.cursor()

for feature in features:
    zone_code = feature['properties']['ZONE_CODE']
    geometry = json.dumps(feature['geometry'])
    cur.execute("SELECT ST_AsText(ST_GeomFromGeoJSON(%s))", (geometry,))
    wkt_geometry = cur.fetchone()[0]

    # Check for null values in zone_code and wkt_geometry
    error_code = None
    if zone_code is None:
        error_code = 'ZoneError'
    elif wkt_geometry is None:
        error_code = 'GeomError'
    
    cur.execute("INSERT INTO MPLSZoning (zone_code, geometry, error_code) VALUES (%s, %s, %s)", (zone_code, wkt_geometry, error_code))
    conn.commit()

cur.close()
conn.close()
